EX01. Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un Pandas Dataframe. Explora les dades que conté, explica breument quines variables hi ha i queda’t únicament amb les columnes que consideris rellevants. Justifica la teva elecció.

In [ ]:
import pandas as pd
from tabulate import tabulate

df = pd.read_csv('DelayedFlights.csv')
print('\nExploración de datos originales:\n')
print(tabulate(df.head(10), headers = 'keys', stralign="center", numalign="right", showindex=False))
print('\nExploración de columnas originales:\n')
print(df.info(verbose=True, show_counts=True))

Junto a la columna que mide el tiempo de retraso en la llegada, se seleccionan las columnas que indican las causas de los retrasos (las 5 últimas), y tres temporales (día de la semana, día del mes y mes), además de UniqueCarrier, que identifica la aerolínea responsable, así como la distancia de los vuelos. FlightNum y TailNum podrían ser necesarias para un análisis más detallado, pero no de forma general. Year se descarta porque es siempre 2008, las variables relacionadas con el retraso o tiempos en pista porque de alguna manera ya están implícitas en ArrDelay, las cancelaciones o desvíos de vuelos, aunque podrían ser útiles de manera complementaria, no nos sirven para estudiar los retrasos.

+ Unnamed: 0: Índice o una columna de identificadores.

Year: Año del vuelo.

+ Month: Mes del vuelo.

+ DayofMonth: Día del mes en que ocurrió el vuelo.

+ DayOfWeek: Día de la semana en que ocurrió el vuelo.

DepTime: Hora real de salida del vuelo (en formato 24 horas).

CRSDepTime: La hora de salida programada ("Computer Reservation System") del vuelo.

ArrTime: La hora real de llegada del vuelo.

CRSArrTime: La hora de llegada programada del vuelo.

+ UniqueCarrier: Un código de identificación para la aerolínea que operó el vuelo.

FlightNum: El número de vuelo.

TailNum: Número de registro de la aeronave que operó el vuelo.

ActualElapsedTime: El tiempo total que tomó el vuelo, desde que el avión inicia su movimiento hasta que vuelve a estar detenido tras aterrizar (en minutos).

CRSElapsedTime: El tiempo de vuelo esperado o programado, de acuerdo con las reservas del sistema (en minutos).

AirTime: La cantidad de tiempo que el avión estuvo en el aire (en minutos).

+ ArrDelay: Retraso en la llegada (en minutos) calculado como la diferencia entre la hora de llegada programada y la hora real de llegada.

DepDelay: Retraso en la salida (en minutos) calculado como la diferencia entre la hora de salida programada y la hora real de salida.

Origin: Código del aeropuerto de origen.

Dest: Código del aeropuerto de destino.

+ Distance: Distancia entre los aeropuertos de origen y destino.

TaxiIn: Tiempo que tardó el avión en llegar a la puerta después de aterrizar.

TaxiOut: Tiempo que tardó el avión en despegar desde la puerta.

Cancelled: Indicador de si el vuelo fue cancelado (1 para cancelado y 0 para no cancelado).

CancellationCode: Código que indica la razón de la cancelación del vuelo.

Diverted: Indicador de si el vuelo fue desviado (1 para desviado y 0 para no desviado).

+ CarrierDelay: Retraso (en minutos) atribuido a la aerolínea.

+ WeatherDelay: Retraso causado por condiciones meteorológicas.

+ NASDelay: Retraso causado por problemas del Sistema Nacional de Espacio Aéreo (NAS).

+ SecurityDelay: Retraso causado por problemas de seguridad.

+ LateAircraftDelay: Retraso causado por un avión que llega tarde.

In [ ]:
# Comprobación de valores nulos

valores_nulos = df[df[['ArrDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']]
         .isnull().all(axis=1)][['ArrDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
                                 'LateAircraftDelay', 'Diverted', 'Cancelled']].sample(15)
print(tabulate(valores_nulos, headers = 'keys', stralign="center", numalign="right", showindex=False))
print('\n')
print(df[((df['ArrDelay'].isnull()) & (df['Diverted'] == 0))][['ArrDelay', 'Diverted', 'Cancelled']].value_counts(dropna=False))
print('\n')
print(df[df['ArrDelay'].isnull()][['ArrDelay', 'Diverted', 'Cancelled']].value_counts(dropna=False))
print('\n')
print(df[((df['ArrDelay'].isnull()) & (df['Diverted'] == 0) & (df['Cancelled'] == 0))][['ArrDelay', 'Diverted', 'Cancelled']])

En la primera tabla vemos como los vuelos sin información de retrasos parecen coincidir con aquellos que o bien han sido desviados o bien han sido cancelados.
En la segunda tabla comprobamos como todos los vuelos sin información sobre retrasos (valores nulos) que no han sido desviados (diverted = 0) coinciden con vuelos cancelados (cancelled = 1), de otro modo tendríamos una fila para cancelled = 0.
Y efectivamente en la última tabla vemos el conteo de vuelos con retrasos nulos, que o bien fueron desviados (7754) o bien fueron cancelados (633).
Finalmente comprobamos como no hay ningún vuelo sin información de retrasos que no fuera desviado o cancelado, al obtener un dataframe vacío (Empty DataFrame).

In [ ]:
df = df[['ArrDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Distance','Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier', 'AirTime', 'Cancelled', 'Diverted']].sample(200000)
df = df[df['ArrDelay'].notnull()]
df.reset_index(drop=True, inplace=True)
print('\nColumnas seleccionadas:\n')
print(tabulate(df.info(verbose=True, show_counts=True)))

Seleccionamos una muestra de 200.000 registros y eliminamos todos aquellos que no tienen información sobre el retraso (ArrDelay), en tanto ya sabemos que en todos los casos se explica por un desvío o cancelación del vuelo, y por lo tanto no tendría sentido utilizar esos valores ni modificarlos de ninguna manera.

EX02_A. Resumeix estadísticament el dataset i les columnes d’interès. Fes una anàlisi estadístic del que consideris rellevant.

In [90]:
# Análisis de factores principales promediables

promediables =  ['ArrDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Distance']

print('\nColumnas de interés promediables: centralización y dispersión\n')
pd.set_option('display.max_columns', 10)
print(df[promediables].describe().round(2))

# Comprobación de retrasos más repetidos

print('\nFrecuencias absolutas de los retrasos:')
print('\n'+str(df['ArrDelay'].value_counts())+'\n')


Columnas de interés promediables: centralización y dispersión

        ArrDelay  CarrierDelay  WeatherDelay   NASDelay  SecurityDelay  \
count  199108.00     128680.00     128680.00  128680.00      128680.00   
mean       42.24         19.22          3.65      15.10           0.09   
std        56.94         43.58         21.38      34.23           2.18   
min       -62.00          0.00          0.00       0.00           0.00   
25%         9.00          0.00          0.00       0.00           0.00   
50%        24.00          2.00          0.00       2.00           0.00   
75%        56.00         21.00          0.00      15.00           0.00   
max      1583.00       1552.00        878.00    1357.00         254.00   

       LateAircraftDelay   Distance  
count          128680.00  199108.00  
mean               25.35     765.33  
std                42.28     576.87  
min                 0.00      24.00  
25%                 0.00     338.00  
50%                 8.00     606.00  
75%

Análisis estadístico del dataset y las columnas principales

Lo primero es situar el dataset, hablamos de retrasos en vuelos domésticos en Estados Unidos durante el año 2008.

Si observamos los parámetros de centralización y dispersión, vemos que la media de los retrasos por vuelo se sitúa en 42,13', y la mediana en 25', y al mismo tiempo los valores más repetidos están entre los 6 y los 10 minutos, siendo la desviación típica de 56.58'.

In [91]:
print('\nCorrelaciones')
print(df[promediables].corr().round(2))


Correlaciones
                   ArrDelay  CarrierDelay  WeatherDelay  NASDelay  \
ArrDelay               1.00          0.50          0.26      0.39   
CarrierDelay           0.50          1.00         -0.07     -0.12   
WeatherDelay           0.26         -0.07          1.00     -0.00   
NASDelay               0.39         -0.12         -0.00      1.00   
SecurityDelay          0.00         -0.02         -0.01     -0.01   
LateAircraftDelay      0.48         -0.18         -0.05     -0.12   
Distance              -0.03          0.03         -0.02      0.02   

                   SecurityDelay  LateAircraftDelay  Distance  
ArrDelay                    0.00               0.48     -0.03  
CarrierDelay               -0.02              -0.18      0.03  
WeatherDelay               -0.01              -0.05     -0.02  
NASDelay                   -0.01              -0.12      0.02  
SecurityDelay               1.00              -0.02      0.01  
LateAircraftDelay          -0.02               1

Por otro lado se observan las correlaciones de los retrasos de los vuelos con sus causas (primera fila de correlaciones), y podemos ver que ni los retrasos por motivos de seguridad ni la distancia presentan correlación con los retrasos, por lo tanto, estos se explican, en este orden por: aerolíneas (0.50), vuelos precedentes (0.48), navegación y espacio aéreo (0.39) - correlación media en los tres casos - y condiciones climatológicas (0.27) - correlación baja.

In [92]:
# Muestra de retrasos no promediables según marco temporal y aerolínea

print('Media de retrasos según marco temporal\n')
print(df[['ArrDelay', 'Month']].groupby('Month').mean().round(2))
print('\n')
print(df[['ArrDelay', 'DayOfWeek']].groupby('DayOfWeek').mean().round(2))
print('\n')
print(df[['ArrDelay', 'DayofMonth']].groupby('DayofMonth').mean().round(2))

Media de retrasos según marco temporal

       ArrDelay
Month          
1         42.67
2         45.07
3         42.11
4         38.42
5         37.43
6         45.82
7         45.47
8         42.59
9         36.21
10        31.54
11        37.62
12        49.93


           ArrDelay
DayOfWeek          
1             42.16
2             44.20
3             40.21
4             40.98
5             43.40
6             39.39
7             44.63


            ArrDelay
DayofMonth          
1              41.90
2              42.12
3              39.28
4              48.23
5              40.43
6              43.39
7              40.81
8              45.11
9              39.12
10             46.85
11             41.94
12             39.13
13             41.65
14             39.97
15             41.34
16             40.07
17             38.62
18             39.26
19             43.85
20             41.52
21             42.84
22             44.79
23             44.18
24             39.28
25    

Por otro lado tenemos los parámetros temporales, que nos muestran las medias de retrasos según el día del mes, el mes y el día de la semana. Las medias están entre 31,54' y 52,45' en todos los casos. Se observa que siempre es el último (día, día de la semana y mes), el que presenta una media de retrasos más alta y los días precedentes medias más bajas.

In [ ]:
print('\nMedia de retrasos por aerolínea:\n')
print(df[['ArrDelay', 'UniqueCarrier']].groupby('UniqueCarrier').mean().round(2).sort_values(by='ArrDelay', ascending = False))

# Vuelos operados por aerolínea

print('\nVuelos totales operados por aerolínea:\n')
print(df['UniqueCarrier'].value_counts())
print('\n')

Respecto a las aerolíneas, las medias de retrasos se mueven entre 23,75' - AQ (mínimo) y 55,98' - B6 (máximo). El número de vuelos no desviados o cancelados por aerolínea dentro de nuestra muestra, entre 63 (AQ) y 39119 (WN). WN, Southwest Airlines sorprende positivamente siendo la tercera con una media de retrasos más baja (29.90') y la que más vuelos opera.

EX02_B. Troba quantes dades faltants hi ha per columna.

In [ ]:
print('Datos faltantes por columna:')
print('\n'+str(df.isnull().sum())+'\n')
print(df[df['CarrierDelay'].isnull()])

Vemos 70202 vuelos con un retraso registrado pero sin información para las causas. Imprimimos los vuelos para explorar los datos y en las columnas seleccionadas no disponemos de ninguna variable adicional que nos permita encontrar un motivo. En cualquier caso el cálculo de las correlaciones no varía, como tampoco lo hacen los parámetros de centralización o dispersión, que descuentan los valores nulos.

EX02_C. Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...).

In [ ]:
# Velocidad media de los vuelos

print('Velocidad media:\n')
df['AvgSpeed'] = df['Distance']/(df['AirTime']/60)
print('\n'+str(df[['Distance', 'AirTime', 'AvgSpeed']])+'\n')

# Retraso sí o no

print('Retraso sí o no:\n')
df['Delay'] = df['ArrDelay'] > 0
print(df[['ArrDelay', 'Delay']].sort_index())

# Proporción del retraso sobre el total causado por la aerolínea

print('\nResponsabilidad de la aerolínea (en tanto por 1):\n')
df['CarrierResponsability'] = df['CarrierDelay'] / df['ArrDelay']
df['CarrierResponsability'] = df['CarrierResponsability'].round(2)
df['CarrierResponsability'] = pd.Series(df['CarrierResponsability'])
print(df['CarrierResponsability'].value_counts())

# Proporción del retraso sobre el total causado por gestión del espacio aéreo (NAS)

print('\nResponsabilidad del sistema nacional del espacio aéreo:\n')
df['NASResponsability'] = df['NASDelay'] / df['ArrDelay']
df['NASResponsability'] = df['NASResponsability'].round(2)
df['NASResponsability'] = pd.Series(df['NASResponsability'])
print(df['NASResponsability'].value_counts())
print('\n')


EX02_D. Fes una taula de les aerolínies amb més endarreriments acumulats.

In [95]:
# Aerolíneas con más vuelos con retraso acumulados

df['Delay'] = df['ArrDelay'] > 0

print('Aerolíneas con más retrasos:\n')
print(df[df['Delay'] == True][['UniqueCarrier']].groupby('UniqueCarrier').value_counts().sort_values(ascending = False))
print('\n')


Aerolíneas con más retrasos:

UniqueCarrier
WN    33686
AA    17686
MQ    13449
UA    12843
OO    12719
DL    10451
XE     9642
US     8580
CO     8579
EV     7776
NW     7375
FL     6842
YV     6406
OH     5007
B6     4957
9E     4775
AS     3535
F9     2701
HA      747
AQ       59
dtype: int64




EX02_E. Quins són els vols més llargs? I els més endarrerits? Busca les rutes més llargues i les que acumulen més retards.

In [ ]:
# Vuelos más largos

print('\nVuelos más largos (valores no repetidos y en millas):\n')
distancias_unicas = pd.Series(df['Distance'].unique())
distancias_unicas = distancias_unicas.sort_values(ascending=False)
distancias_unicas.reset_index(drop=True, inplace=True)
distancias_unicas.index = distancias_unicas.index + 1
print(distancias_unicas.head(10))

# Vuelos más atrasados

print('\nAtrasos máximos (en minutos):\n')
atrasos = df[['ArrDelay', 'UniqueCarrier', 'DayofMonth', 'Month', 'DayOfWeek']].sort_values(by='ArrDelay', ascending=False).head(20)
atrasos.reset_index(drop=True, inplace=True)
atrasos.index = atrasos.index + 1
print(atrasos)


Comprobamos como los 20 primeros vuelos con retrasos más largos se los reparten prácticamente todos entre tres aerolíneas: NW, AA y MQ, destacando en este caso NW negativamente.
Las fechas no muestran ningún patrón aparente, ni en los días, meses o días de la semana.

EX03. Exporta el dataset net i amb les noves columnes a Excel

In [ ]:
print('Dataset que se exporta:\n')
df.reset_index(drop=True, inplace=True)
print(df.info())
df.to_excel('DelayedFlightsAdjusted.xlsx')